In [1]:

import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%load_ext autoreload
%autoreload 2
%matplotlib qt

# O método de Newton

## $ \S 1 $ Descrição do método de Newton

Suponha que desejemos encontrar um zero de uma função *diferenciável* $ f $. Partindo de uma estimativa inicial $ x_0 $, a idéia por trás do **método de Newton** é substituir a função pela sua reta tangente na estimativa atual $ x_n $ e tomar a  intersecção desta com o eixo-$x$ como o valor da próxima estimativa $ x_{n + 1} $. 

Em símbolos, a reta tangente ao gráfico de $ f $ em $ x_n $ é descrita pela equação:
$$
y = f(x_n) + f'(x_n)\,(x - x_{n})\,.
$$
Fazendo $ y = 0 $, encontramos que a aproximação $ x_{n + 1} $ seguinte é dada por
$$
\boxed{x_{n + 1} = x_n - \frac{f(x_n)}{f'(x_n)}}
$$

## $ \S 2 $ Vantagens e desvantagens do método de Newton

No método de Newton o erro cometido na iteração seguinte é aproximadamente proporcional ao *quadrado* do erro da iteração atual. Portanto podemos esperar que uma vez que o erro seja menor que $ 0.1 $, a cada passo o número de dígitos decimais de precisão dobrará. Este desempenho deve ser comparado com o do método da bisseção, em que o erro seguinte é aproximadamente proporcional ao erro atual (por um fator de $ 1/2 $). Informalmente, isto significa que uma vez que consigamos uma estimativa próxima o suficiente de um zero, o método de Newton convergirá muito mais rapidamente que os outros métodos que estudamos. Outra vantagem é que para aplicar o método de Newton, não precisamos encaixotar o zero.

Entretanto, o método de Newton tem duas desvantagens significativas:
* Ele exige o cálculo da derivada da função à qual será aplicado.
* Ele nem sempre funciona, ou seja, a seqüência $ (x_n) $ pode não convergir.

Por estes motivos uma estratégia mais adequada é utilizar o método de Newton em combinação com um mais confiável, como o método da bissecção.

**Problema 1:** Sem usar o computador, aplique o método de Newton para aproximar um zero das funções abaixo, usando a estimativa inicial $ x_0 $ e o número $ N $ de iterações  indicados:

(a) $ y = xe^x - 1\, ,\quad  x_0 = \frac{1}{2}\, , \quad  N = 3\, $. 

(b) $ y = \arctan x\, ,\quad  x_0 = 1\, , \quad  N = 4\, $. 

(c) $ y = \ln x - 3\, ,\quad  x_0 = 10\, , \quad  N = 4\, $. 

## $ \S 3 $ Fórmula para o erro no método de Newton

**Teorema 3.1: (Teorema do valor médio para integrais, versão estendida):** _Sejam $ f $ e $ g $ funções contínuas $ [a, b] \to \mathbb{R} $ tais que $ g $ não troca de sinal em $ [a, b] $. Então existe $ c \in [a, b] $ tal que_
$$
\int_a^b f(x)\,g(x)\,dx = f(c)\int_a^bg(x)\,dx
$$

Quando estudamos interpolação polinomial, vimos que, para cada $ x $ em $ [a, b] $,
$$
f(x) - p(x) = \frac{1}{2}(x-a)(x-b)f''(c_x)\qquad \text{para algum $ c_x $ em $ (a, b) $}.
$$
Escrevendo $ (x-a)(x-b) = g(x) $, segue que
$$
E = \frac{1}{2}\int_a^b g(x)f''(c_x)\,dx\,.
$$

Sejam
$$
    m = \min_{[a, b]} f''\quad \text{e} \quad M = \max_{[a, b]} f''.
$$
Como $ g \le 0 $ em $ [a, b] $, para qualquer $ x \in [a, b] $ temos:
\begin{equation*}
    Mg(x) \le g(x) f''(c_x) \le m g(x)\,.
\end{equation*}
Integrando, deduzimos que
\begin{equation*}
M \int_a^b g(x)\,dx \le \int_a^b g(x)f''(c_x)\,dx \le m \int_a^b g(x)\,dx .
\end{equation*}

Agora, novamente como $ g \le 0 $ em $ [a, b] $, se dividirmos todos estes termos por $ \int_a^b g(x)\,dx $, que é negativa, precisamos inverter os sentidos das desigualdades. Assim:
\begin{equation*}
    m \le \frac{\int_a^b g(x)f''(c_x)\,dx}{\int_a^b g(x)\,dx} \le M .
\end{equation*}
Como $ f'' $ é contínua por hipótese e $ m $ e $ M $ são seus valores extremos no intervalo $ [a, b] $, pelo Teorema do Valor Intermediário existe um $ c \in [a, b] $ tal que o termo do meio acima é igual a $ f''(c) $. Equivalentemente,
\begin{equation*}
    E = \frac{1}{2}\int_a^b g(x)f''(c_x)\,dx = \frac{f''(c)}{2}\int_a^b g(x)\,dx \qquad (\text{para algum } c \in [a, b]).
\end{equation*}

In [6]:

def print_solution(xs: list[float], ys: list[float], freq: int = 1) -> None:
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first and last line are always printed. If freq == 0, then only these
          lines are printed.
    Output: None.
    Prints: A header and the table described above.
    """
    def print_header() -> None:
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n: int, x: float, y: float) -> None:
        """
        Pretty-prints n, x and y.
        """
        print(f"|      {n:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    print_line(0, xs[0], ys[0])
    for n in range(1, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [7]:
def newton_animation(f: Callable[[float], float],
                     df: Callable[[float], float],
                     a: float, b :float, x: float,
                     N: int = 4, title: str = "", duration: float = 0.75
                     ) -> tuple[list[float], list[float]]:
    """
    Displays an animation of Newton's method applied to a function.
    Parameters:
        * A differentiable real function f.
        * Its derivative df (as a function).
        * The two endpoints a and b of the interval in the x-axis where the
          animation takes place.
        * An initial estimate x for the zero in [a, b].
        * The maximum number N of iterations.
        * A title to be displayed at the top of the diagram.
        * The duration of the pause between slides of the animation, in seconds.
          Set duration = 0 to produce a figure instead of an animation.
    Output:
        * Two lists xs and ys containing the estimates and the values of the
          function f at each of them.
    Displays:
        * The animation in a pop-up window.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    %matplotlib qt
    
    
    def pause(duration):
        """
        Pauses the animation for duration seconds, provided duration > 0.
        """
        if duration > 0:
            plt.pause(duration)

    def tangent_line_at(x_0):
        return lambda x: f(x_0) + df(x_0) * (x - x_0)
    
    def iteration(x):
        return x - f(x) / df(x)


    cmap = plt.get_cmap("tab10")               # Used to control the colors.
    P = 200                                    # Number of points in each plot.
    width = 1.75                               # Line width.
    marker_size = 5
    domain = np.linspace(a, b, P)              # Generates P nodes from a to b.
    xs = [x]                                   # Stores the estimates.
    for _ in range(N):                         # Filling xs.
        xs.append(iteration(xs[-1]))
    ys = [f(x) for x in xs]                    # Stores f of the estimates.
    tangent_lines = [tangent_line_at(x) for x in xs]

    # Generate sample points for x-intervals between consecutive estimates:
    xs_range = [np.linspace(xs[n], xs[n + 1], P) for n in range(N)]
    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N)]

    # Draw the graph of f:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=width)
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.title(title)
    plt.grid(True)
    plt.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    plt.legend()

    # Mark the initial estimate on the x-axis:
    pause(duration)
    plt.plot(xs[0], 0, color=cmap(1), marker="x",
             mew=width, label=f'$ x_{0} $')

    for n in range(0, N):
        # Mark x_n on the x-axis:
        plt.plot(xs[n], 0, color=cmap(n + 1), marker="x", mew=width)
        pause(duration)
        # Draw the segment of the line x = x_n from y = 0 to y = y_n:
        plt.plot(xs_vert[n], ys_vert[n], linestyle='dotted',
                 lw=width, color='black')
        # Plot (x_n, y_n):
        plt.plot(xs[n], ys[n], color='black', marker="o", ms=marker_size)
        pause(duration)
        # Plot the tangent line at (x_n, f(x_n)):
        plt.plot(xs_range[n], tangent_lines[n](xs_range[n]),
                 linestyle='--', color=cmap(n + 2))
        pause(duration)
        plt.plot(xs[n + 1], 0, color=cmap(n + 2), marker="x",
                 mew=width, label=f'$ x_{n + 1} $')
        plt.legend()
    
    return xs, ys

from numpy import arctan, sin, cos, exp
a = -1.1
b = 1.1
N = 4
f = lambda x: 2 * x**5 - x + 1
df = lambda x: 10 * x**4 - 1
x_0 = 0.0
titulo = "Método de Newton para $ y = 2x^5 - x + 1 $."\
         "\nPerto do mínimo local, a estimativa é jogada para longe."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, 0.75)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.00000000         1.00000000   |
|      01         1.00000000         2.00000000   |
|      02         0.77777778         0.79147826   |
|      03         0.48017397         0.57087924   |
|      04         1.69898988        27.61388409   |
|_________________________________________________|



![Exemplo de comportamento inadequado do método de Newton perto de extremos
locais](fig_2-6_exemplo_2.png "Exemplo de comportamento inadequado do método de
Newton perto de extremos locais")